## Create Maps with Sensors, Polygons and Grid Data

In [1]:
import folium
import numpy as np
from constants import Constants
from mapping import Mapping
from grid_definition import GridDefinition
import shapely
import json

grid_definition = GridDefinition()
grid_definition.init()
constants = Constants()
GRID_SIZE = constants.getGridSize()
print(GRID_SIZE)
constants = Constants()
mapping = Mapping()
#map_dir = <DIRECTORY_TO_SAVE_MAP_IMAGES>

60


## A Map with Leon ZBE1, ZBM, existing sensors, and study area outline

In [2]:
mapCenter = [grid_definition.getCenterLat(), grid_definition.getCenterLon()]
leon_overview_map = folium.Map(location=mapCenter, zoom_start=13,tiles=None, control_scale=True)
folium.TileLayer('Stamen Toner', name='Leon Overview').add_to(leon_overview_map)

feature_group1 = folium.FeatureGroup(name='Zona Baja Emission Polygon 1')
feature_group2 = folium.FeatureGroup(name='Zona Baja Motorizacion Polygon')
feature_group3 = folium.FeatureGroup(name='Proposed Study Area')
feature_group4 = folium.FeatureGroup(name='Existing Sensors')

ZBE_polygon_path = '../osm/ZBE_P1.geojson'
with open(ZBE_polygon_path) as f:
            ZBE = json.load(f)      
        
mapping.plotPolygon(feature_group1, ZBE, '#FF0D00')
leon_overview_map.add_child(feature_group1)

#mapping.plotPolygon(leon_overview_map, ZBE, '#FF0D00').add_to(feature_group1)

ZBM_polygon_path = '../osm/Zona_Baja_Motorizacion.geojson'
with open(ZBM_polygon_path) as f:
            ZBM = json.load(f)
mapping.plotPolygon(feature_group2, ZBM, '#FF8500')
leon_overview_map.add_child(feature_group2)

cells = mapping.getGeoCells()

for cell_num, cell in enumerate(cells):
    cell_multi = cell[0]["geometry"]
    cell_shape = shapely.geometry.asShape(cell_multi)
    row = int(cell_num / GRID_SIZE)
    col = int(cell_num % GRID_SIZE)
    if (row == 0 or row == (GRID_SIZE - 1) or col == 0 or col == (GRID_SIZE - 1)):
    #index = int(land_use[row][col])
        mapping.plotPolygon(feature_group3, cell_multi, '#808080')
        leon_overview_map.add_child(feature_group3)

    
mapping.mapExistingSensors(feature_group4, 'black')
leon_overview_map.add_child(feature_group4)

leon_overview_map.add_child(folium.map.LayerControl())
leon_overview_map.save("../images/maps/LeonSmallOverview.html")

#leon_overview_map

In [ ]:
##Map the stationary sensors
stationary_map = folium.Map(location=[55.943, -3.19], zoom_start=15,tiles="Stamen Toner")
mapStationarySensors(stationary_map)

#stationary_map.save(map_dir + "stationarySensors.html")

In [ ]:
## Map a data pollution collection walk
mobile_map = folium.Map(location=[55.943, -3.19], zoom_start=15,tiles="Stamen Toner")
sid = 'XXM008' #or 'XXM007'. ID of the pollution belt that data was collected with
mapPollutionWalk(mobile_map, start_date, sid)

#mobile_map.save(map_dir + str(start_date) + "_" + str(sid_plot) + ".html")

In [ ]:
def mapStationarySensors(map_obj):
    staticCoords = constants.getStaticCoords()
    for i in range(staticCoords.shape[0]):
        folium.CircleMarker((staticCoords[i][0], staticCoords[i][1]),
                   radius=8,
                    color='#000000',
                    weight=1.0,
                    fill_color='#000000',
                    fill=True,
                    fill_opacity=1,
                   ).add_to(map_obj)

def mapPollutionWalk(mapObj, start_date, sid):
    end_date = getEndDate(start_date)
    data_dir = "/Users/zoepetard/Google Drive/Edinburgh/MscProj/FillingTheGaps/data/raw/personal/"+str(start_date)+"-"+str(end_date)+"/"
    sids = ['XXM007', 'XXM008']
    pdata = data_downloader.readAirSpeckPCSV(start_date, end_date, data_dir)
    belt_index = sids.index(sid)
    
    maxPM = np.max(pdata[belt_index]["PM2.5"])
    minPM = np.min(pdata[belt_index]["PM2.5"])
            
    ##Add validation walk
    for j in range(len(pdata[belt_index])):
        folium.CircleMarker((pdata[belt_index]["latitude"][j], pdata[belt_index]["longitude"][j]),
                    radius=5,
                    color='#000000',
                    weight=1.0,
                    fill_color=osm_reader.assignColor(pdata[belt_index]["PM2.5"][j], maxPM, minPM),
                    fill=True,
                    fill_opacity=1.0,
                   ).add_to(mapObj)

def getEndDate(start_date):
    return start_date + 1